In [1]:
using StatsBase, Compat, Distributions, LinearAlgebra

#creates structure individuals for indviduwas investing in two tasks, A and B. 'cheat' parameter was created but never used
struct unit
           Viability
           Fecundity
           cheat
       end

#create initial population consisting of n = 'size' individuals
function create_units(size, var)
    unit_list = unit[]
    for i = 1:size
        if var == "rand"
            v = rand()
        elseif var == "equal"
            v = .5
        elseif var == "alt"
            v = (i+1)%2
        end
        f = 1-v
        push!(unit_list, unit(v,f, "coop"))
    end
    return unit_list
end

create_units (generic function with 1 method)

In [2]:
# calculates fitness, as per the hybrid model (one parameter is shared across interactions)
function fitness(unit_list, c, pow)
    fitness = 0
    for i = 1:length(unit_list)
        Aa = 0
        for j = 1:length(unit_list)
            Aa += ((unit_list[j].Viability)) * c[i,j]
        end
        fitness += (Aa * (unit_list[i].Fecundity))^pow
    end
    return fitness
end

fitness (generic function with 1 method)

In [3]:
# Wright-Fisher evolution.
function evolve3(units, c, pop_size, steps, fit_pow, int_strength, c_mod)
    # probability that ONE member of given group will mutate to a random distribution
    mutate_prob = .02
    # going to fill pop with (group, c) tuples
    pop = []
    fitness_rank = []
    mean_spc = 0
    evolving_c = c #Matrix{Float64}(I,length(c[:,1]),length(c[1,:])) # nxn identity matrix
    base_c = unnormalize(copy(c)) # nxn matrix with 1's where interactions are allowed, zeros elsewhere

    for i = 1:pop_size
        push!(pop, (copy(units), c))
    end

    altC = create_fancy_c(10, "nn", 0, int_strength)
    for i = 1:steps
        altC = create_fancy_c(10, "nn", (i%2)*1, int_strength)

        # individual mutations
        for j = 1:pop_size
            for l = 1:length(units)
                if rand(0:1000)/1000 < mutate_prob
                    rando = rand(1:length(units))
                    # choose new value from gaussian centered at pop[j][rando].Viability,
                    # with standard deviation of .1. Use of truncation is obvious.
                    # Final parameter in rand is the number of values to generate
                    # this mess produces a value in a list, so the trailing braced index retrieves the value
                    new_value = rand(Truncated(Normal(pop[j][1][l].Viability, .1), 0, 1), 1)[1]
                    # new_value = rand(0:1000)/1000
                    # splice!(pop[j], rando, [unit(new_value, 1-new_value, "coop")])
                    pop[j][1][l] = unit(new_value, 1-new_value, "coop")
                end
            end
            # mutating c can be toggled off by commenting out the following loop AND changing evolving_c to c in the initialization of pop:
             if rand(0:1000)/1000 < mutate_prob
                 pop[j] = (pop[j][1], mutate_c(base_c, pop[j][2]))
             end
        end

        # update fitness_rank
        new_fitness_rank = Float64[]
        for m = 1:pop_size
            push!(new_fitness_rank, fitness(pop[m][1], pop[m][2], fit_pow))
        end
        fitness_rank = new_fitness_rank

        # fitness-weighted selection
        new_pop = wsample(pop, fitness_rank/pop_size, pop_size)
        pop = []
        # this copying step is necessary because wsample keeps underlying groups linked instead of copying groups from pop to populate new_pop
        for i = 1:pop_size
            push!(pop, (copy(new_pop[i][1]),(copy(new_pop[i][2]))))
        end
    end

    # calculate mean 'specialization' for all groups in final round.
    # 0 indicates no specialization, 1 indicates total division of labor; linear scale from 0 to 1
    sum = 0
    for j = 1:length(pop_size)
        for i = 1:length(units)
            sum += 2*(max(pop[j][1][i].Viability, pop[j][1][i].Fecundity)-.5)
        end
    end
    mean_spc = sum/length(units)/length(pop_size)

    # update fitness_rank (for final step)
    final_fitness_rank = Float64[]
    for m = 1:pop_size
        push!(final_fitness_rank, fitness(pop[m][1], altC, fit_pow))
    end

    # calculate mean specialization for fittest group in final round
    index_of_max = findmax(final_fitness_rank)[2]
    max_sum = 0
    for i = 1:length(units)
        max_sum += 2*(max(pop[index_of_max][1][i].Viability, pop[index_of_max][1][i].Fecundity)-.5)
    end
    max_spc = max_sum/length(units)

    return [0, 0, 0, 0, max_spc, mean_spc]
    # return pop[index_of_max][2]
end

evolve3 (generic function with 1 method)

In [4]:
function create_fancy_c(size, topology, param, strength)
    new_mat = Matrix{Float64}(I,size,size)
    
    if topology == "nn"
        for i = 1:param
            new_mat[1,size] = 1.0
            new_mat[size,1] = 1.0
            for j = 1:size-i
                new_mat[i+j,j] = 1.0
                new_mat[j,i+j] = 1.0
            end
        end

    
    elseif topology == "rand_n"
        tries = 0
        adds = param*size
        for i = 1:size
            j = rand(1:size)
            while sum(new_mat, 1)[i] < (param + 1)
                if sum(new_mat, 1)[j] < (param + 1)
                    new_mat[i, j] = new_mat[j, i] = 1
                end
                j = rand(1:size)
                tries += 1
                if tries > 100
                    return create_fancy_c(size, topology, param, strength)
                end

            end
        end

    # create any 'special' topology by manually adding connections (symmetric)
    elseif topology =="special"
        connections = [(1, 2), (2,3), (1,3), (3,4), (4,5), (5, 6), (7,8), (8,9), (9,10)]
        for i = 1:length(connections)
            new_mat[connections[i][1], connections[i][2]] = new_mat[connections[i][2], connections[i][1]] = 1
        end


    elseif topology == "spc_optimized"
        for i = 1:size
            for j = 1:size
                if i%2 == 0 && j%2 == 1
                    new_mat[i,j] = 1.0
                    new_mat[j,i] = 1.0
                end
            end
        end
    end


    # distributes resources across connections
    new_new_mat = copy(new_mat)
    sum_rows = sum(new_new_mat, dims=2)
    for i = 1:size
        for j = 1:size
            if i != j && new_mat[j, i] == 1
                new_new_mat[j, i] = strength/sum_rows[i]
            elseif i == j
                new_new_mat[j, i] = 1-strength + strength/sum_rows[i]
            end
        end
    end

    return new_new_mat
end

create_fancy_c (generic function with 1 method)

In [27]:
function param_sweep()
    # parameters
    integration_strengths = collect(0:.1:1)
    specialization_powers = collect(.5:.1:1.9)
    pop_size = 1000
    steps = 1000
    units = create_units(10,"rand")
    c_mod = 0
    topology = "spc_optimized" # "nn" for nearest neighbors, "special" for manually chosen interactions, and "spc_optimized" for the alternating topology that maximizes the benefits of specialization

    results = []
    # create output labels
    push!(results, ["integration_strength", "specialization power", "null", "null", "null", "null", "max specialization", "specialization"])
    for i = 1:length(integration_strengths)

        # create new network with each interaction strength
        network = create_fancy_c(10, topology, c_mod, integration_strengths[i])

        for j = 1:length(specialization_powers)
            point = evolve3(units, network, pop_size, steps, specialization_powers[j], integration_strengths[i], c_mod)
            betas = Array{Float64}(1:10)
            for betas_index = 1:10
                betas[betas_index] = 1-network[betas_index,1]+network[betas_index,2]
            end
            point[1] = findmax(betas)[1]
            # point = invest_plot(units, 100, 100, integration_strengths[i], specialization_powers[j], c_mod)
            push!(results, [integration_strengths[i], specialization_powers[j], point[1], point[2], point[3], point[4], point[5], point[6]])
            println(i)
            println(j)
            println()
        end
    end
    #writecsv("spc_alt_noninteracting_2%_1k_1k.csv", results)
    return results
end

param_sweep (generic function with 1 method)

In [6]:
function mutate_c(base_c, evolving_c)
    # for this first iteration, I'm going to make 'shared' resources be shared equally across connections
    i = rand(1:length(base_c[:,1]))
    # adjust amount of fecundity kept by individuals
    new_value = rand(Truncated(Normal(evolving_c[i,i], .1), 0, 1), 1)[1]
    # new_value = rand(0:1000)/1000
    new_c = copy(evolving_c)
    new_c[i, i] = new_value
    total_shared = 1 - new_value
    for j = 1:length(base_c[:,1])
        if j != i && base_c[i, j] == 1
            new_c[i,j] = total_shared/(sum(base_c[i, :])-1) # -1 because self term is counted in sum_cols_base
        end
    end
    return new_c
end

mutate_c (generic function with 1 method)

In [7]:
function unnormalize(mat)
     # size returns a tuple, so the trailing index grabs an integer
    for i = 1:length(mat[:,1])
        for j = 1:length(mat[1,:])
             if mat[i,j] != 0
                 mat[i,j] = 1
             end
         end
     end
     return mat
 end

unnormalize (generic function with 1 method)

In [8]:
# to be used only when interactions are evolving
function reps(topology)
    # parameters
    reps = 50
    specialization_powers = collect(0.0:.1:1.5)
    pop_size = 200
    steps = 50000
    units = create_units(10,"rand")
    c_mod = 1
    #topology = "spc_optimized" # "nn" for nearest neighbors, "special" for manually chosen interactions, and "spc_optimized" for the alternating topology that maximizes the benefits of specialization

    results = []
    # create output labels
    push!(results, ["rep", "specialization power", "max_beta", "null", "null", "null", "max specialization", "specialization"])


    # create new network with each interaction strength
    network = create_fancy_c(10, topology, c_mod, 1)
    for i = 1:reps
        for j = 1:length(specialization_powers)
            point = evolve3(units, network, pop_size, steps, specialization_powers[j], 1, c_mod)
            betas = Array{Float64}(1:10)
            for betas_index = 1:9
                betas[betas_index] = 1-network[betas_index,betas_index]+network[betas_index,betas_index+1]
            end
            betas[10] = 1 - network[10,10]+network[10,1]
            point[1] = findmax(betas)[1]
            # point = invest_plot(units, 100, 100, integration_strengths[i], specialization_powers[j], c_mod)
            push!(results, [i, specialization_powers[j], point[1], point[2], point[3], point[4], point[5], point[6]])
        end
        println(i)
    end
    #writecsv("evolvingC_spcOptimized_100_1000.csv", results)
    return results
end

reps (generic function with 1 method)

In [9]:
spc = create_fancy_c(10,"spc_optimized",1,1);
1-spc[3,3]+spc[3,4]

1.0

In [28]:
#res_nn = reps("nn");
res_spc = param_sweep();
using DelimitedFiles
writedlm( "spc_optimized_reps.csv",  res_spc, ',');
writedlm( "nn_reps.csv",  res_nn, ',');

1
1

1
2

1
3

1
4

1
5

1
6

1
7

1
8

1
9

1
10

1
11

1
12

1
13

1
14

1
15

2
1

2
2

2
3

2
4

2
5

2
6

2
7

2
8

2
9

2
10

2
11

2
12

2
13

2
14

2
15

3
1

3
2

3
3

3
4

3
5

3
6

3
7

3
8

3
9

3
10

3
11

3
12

3
13

3
14

3
15

4
1

4
2

4
3

4
4

4
5

4
6

4
7

4
8

4
9

4
10

4
11

4
12

4
13

4
14

4
15

5
1

5
2

5
3

5
4

5
5

5
6

5
7

5
8

5
9

5
10

5
11

5
12

5
13

5
14

5
15

6
1

6
2

6
3

6
4

6
5

6
6

6
7

6
8

6
9

6
10

6
11

6
12

6
13

6
14

6
15

7
1

7
2

7
3

7
4

7
5

7
6

7
7

7
8

7
9

7
10

7
11

7
12

7
13

7
14

7
15

8
1

8
2

8
3

8
4

8
5

8
6

8
7

8
8

8
9

8
10

8
11

8
12

8
13

8
14

8
15

9
1

9
2

9
3

9
4

9
5

9
6

9
7

9
8

9
9

9
10

9
11

9
12

9
13

9
14

9
15

10
1

10
2

10
3

10
4

10
5

10
6

10
7

10
8

10
9

10
10

10
11

10
12

10
13

10
14

10
15

11
1

11
2

11
3

11
4

11
5

11
6

11
7

11
8

11
9

11
10

11
11

11
12

11
13

11
14

11
15



UndefVarError: UndefVarError: res_nn not defined

In [29]:
res_spc

166-element Array{Any,1}:
 ["integration_strength", "specialization power", "null", "null", "null", "null", "max specialization", "specialization"]
 [0.0, 0.5, 2.0, 0.0, 0.0, 0.0, 0.0865515, 0.237756]                                                                     
 [0.0, 0.6, 2.0, 0.0, 0.0, 0.0, 0.142254, 0.280777]                                                                      
 [0.0, 0.7, 2.0, 0.0, 0.0, 0.0, 0.124753, 0.256864]                                                                      
 [0.0, 0.8, 2.0, 0.0, 0.0, 0.0, 0.0861639, 0.221587]                                                                     
 [0.0, 0.9, 2.0, 0.0, 0.0, 0.0, 0.0705189, 0.12047]                                                                      
 [0.0, 1.0, 2.0, 0.0, 0.0, 0.0, 0.0636962, 0.177628]                                                                     
 [0.0, 1.1, 2.0, 0.0, 0.0, 0.0, 0.12723, 0.172467]                                                                      

In [ ]:
network = create_fancy_c(10, "spc_optimized", 1, 1)

In [23]:
Array{Float64}(1:10)

10-element Array{Float64,1}:
  1.0
  2.0
  3.0
  4.0
  5.0
  6.0
  7.0
  8.0
  9.0
 10.0